In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [2]:
from azureml.core.model import Model

model = Model(ws, name='bionelessModel_SWR')
transformer = Model(ws, name='transformer_SWR')
translater = Model(ws, name='translater_SWR')
print(model)
print(transformer)
print(translater)

Model(workspace=Workspace.create(name='ws-projet-transverse-bioneless', subscription_id='ba047aef-dd20-494c-9c7e-2d86764842a6', resource_group='rg-projet-transverse-bioneless'), name=bionelessModel_SWR, id=bionelessModel_SWR:1, version=1, tags={}, properties={})
Model(workspace=Workspace.create(name='ws-projet-transverse-bioneless', subscription_id='ba047aef-dd20-494c-9c7e-2d86764842a6', resource_group='rg-projet-transverse-bioneless'), name=transformer_SWR, id=transformer_SWR:1, version=1, tags={}, properties={})
Model(workspace=Workspace.create(name='ws-projet-transverse-bioneless', subscription_id='ba047aef-dd20-494c-9c7e-2d86764842a6', resource_group='rg-projet-transverse-bioneless'), name=translater_SWR, id=translater_SWR:1, version=1, tags={}, properties={})


In [3]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="bioneless_env")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("lightgbm")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="Entry_script.py", environment=myenv)

In [4]:
from azureml.core.webservice import AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

#DEBUG
#from azureml.core.webservice import LocalWebservice
#deployment_config = LocalWebservice.deploy_configuration(port=8890)

In [5]:
service_name = "bioneless-service-swr"
service = Model.deploy(ws, service_name, [model, transformer, translater], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [9]:
scoring_uri = service.scoring_uri
print(scoring_uri)

http://8be0dbc9-8b18-4e2e-b9ce-77c0b7e0b626.westeurope.azurecontainer.io/score


In [7]:
import requests
import json
import pandas as pd

file = "MouvementData-01-03-2021 10-49-28.csv"

headers = {'Content-Type':'application/json'}

test_data = json.dumps([
                       {"0": 1.6927522, "1": 9.265843, "2": -2.238647, "3": 0.0033554828, "4": 0.01798815, "5": 0.0039095376},
                       {"0": -0.40463245, "1": 9.486116, "2": -2.0159793, "3": 0.3659877, "4": -0.069383904, "5": 0.097713396},
                       {"0": -0.40463245, "1": 9.486116, "2": -2.0159793, "3": 0.3659877, "4": -0.069383904, "5": 0.097713396}
                      ])
    
print("## DATA ##")
print(test_data)
print("##########\n")

## DATA ##
[{"0": 1.6927522, "1": 9.265843, "2": -2.238647, "3": 0.0033554828, "4": 0.01798815, "5": 0.0039095376}, {"0": -0.40463245, "1": 9.486116, "2": -2.0159793, "3": 0.3659877, "4": -0.069383904, "5": 0.097713396}, {"0": -0.40463245, "1": 9.486116, "2": -2.0159793, "3": 0.3659877, "4": -0.069383904, "5": 0.097713396}]
##########



In [8]:
response = requests.post(scoring_uri, data=test_data, headers=headers)
print("Status Code : ", response.status_code)
print("Time needed : ", response.elapsed)
print("Prediction :", response.json())

Status Code :  200
Time needed :  0:00:00.079042
Prediction : {"Immobile": "1", "Marcher": "2"}
